In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [2]:
train_data = pd.read_csv('./train_data.csv')
train_data.describe()

,热循环次数,电压(V),电流(A),功率(KW),厚度(μm),氩气(SPLM),氢气(SPLM),载气,送粉量(g/min),喷涂距离(mm),剥落面积百分比
count,105.000000,105.000000,105.000000,105.000000,105.000000,105.0,105.0,105.0,105.000000,105.0,105.000000
mean,210.647619,67.009524,600.476190,40.270476,289.428571,35.0,5.0,300.0,25.809524,120.0,7.039333
std,130.111341,0.814475,40.723763,3.217468,86.988379,0.0,0.0,0.0,1.850651,0.0,4.979992
min,26.000000,66.000000,550.000000,36.300000,200.000000,35.0,5.0,300.0,25.000000,120.0,0.000000
25%,110.000000,66.000000,550.000000,36.300000,200.000000,35.0,5.0,300.0,25.000000,120.0,3.302424
50%,200.000000,67.000000,600.000000,40.200000,300.000000,35.0,5.0,300.0,25.000000,120.0,7.135727
75%,302.000000,68.000000,650.000000,44.200000,300.000000,35.0,5.0,300.0,25.000000,120.0,10.150110
max,505.000000,68.000000,650.000000,44.200000,430.000000,35.0,5.0,300.0,30.000000,120.0,23.890330


In [3]:
def read_data(drop_flag=False):
    train_data = pd.read_csv('./train_data.csv')
    train_y = train_data.values[:, -1]
    if drop_flag:
        train_x = train_data.drop(columns=['剥落面积百分比', '喷涂距离(mm)', '载气', '氢气(SPLM)', '氩气(SPLM)']).values
    else:
        train_x = train_data.drop(columns=['剥落面积百分比']).values
    scaler = StandardScaler()
    train_x = scaler.fit_transform(train_x)
    
    test_data = pd.read_csv('./test_data.csv')
    test_y = test_data.values[:, -1]
    if drop_flag:
        test_x = test_data.drop(columns=['剥落面积百分比', '喷涂距离(mm)', '载气', '氢气(SPLM)', '氩气(SPLM)']).values
    else:
        test_x = test_data.drop(columns=['剥落面积百分比']).values
    test_x = scaler.transform(test_x)
    return train_x, train_y, test_x, test_y
train_x, train_y, test_x, test_y = read_data(drop_flag=False)

# Ridge

In [4]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=12.75)
ridge.fit(train_x, train_y)
print(ridge.score(test_x, test_y))
print(ridge.predict(test_x))

-3.968633593929505
[11.54827606 12.20747852 14.26748623]


# Lasso

In [5]:
from sklearn.linear_model import Lasso
model = Lasso(alpha=0.1)
model.fit(train_x, train_y)
print(model.score(test_x, test_y))
print(model.predict(test_x))

-8.505493158756797
[12.35991267 13.09194711 15.37955474]


# ElasticNet

In [6]:
from sklearn.linear_model import ElasticNet
model = ElasticNet(alpha=1, l1_ratio=0.4, max_iter=100000)
model.fit(train_x, train_y)
print(model.score(test_x, test_y))
print(model.predict(test_x))

0.748028288901369
[ 9.39320097  9.73929762 10.82084964]


# KNeighborsRegressor

In [7]:
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor(n_neighbors=2)
model.fit(train_x, train_y)
print(model.score(test_x, test_y))
print(model.predict(test_x))

-0.8115048521574397
[9.232152 9.232152 9.232152]


# SVR

In [9]:
from sklearn.svm import SVR
model = SVR(C=23)
model.fit(train_x, train_y)
print(model.score(test_x, test_y))
print(model.predict(test_x))

-1.6365274032720727
[9.70684097 9.6633838  8.70102792]


# GradientBoostingRegressor

In [10]:
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor(min_samples_split=7, n_estimators=300)
model.fit(train_x, train_y)
print(model.score(test_x, test_y))
print(model.predict(test_x))

0.2871317468957746
[10.38467302  8.67477266 11.1512662 ]


# RF

In [11]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(max_features=8, n_estimators=76)
model.fit(train_x, train_y)
print(model.score(test_x, test_y))
print(model.predict(test_x))

-0.6007214068606781
[9.31971612 9.28635246 9.38002996]


# XGBoost

In [13]:
from xgboost import XGBRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
model = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             eta=0.3, gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=3, min_child_weight=1,
             monotone_constraints='()', n_estimators=210, n_jobs=16,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)
model.fit(train_x, train_y)
print(r2_score(test_y, model.predict(test_x)))
print(model.predict(test_x))

[20:13:43] WARNING: ../src/learner.cc:573: 
Parameters: { "enable_categorical" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


-2.088745785251392
[8.525156 8.561746 8.680573]


In [14]:
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
model = XGBRegressor(max_depth=3, learning_rate=0.01, n_estimators=700)
model.fit(train_x, train_y)
print(r2_score(test_y, model.predict(test_x)))
print(model.predict(test_x))

0.3839586389045452
[ 9.930362 10.012045 10.477219]
